In [1]:
import urllib.request as urllib
from bs4 import BeautifulSoup

In [2]:
# Read in a webpage
def read_in_page(url):
    #Query the website and return the html to the variable 'page'
    page = urllib.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    return(soup)

# Get all relevant links in a webpage
def get_links(page):
    links = page.find_all("a", class_="action")
    links_new = []
    for link in links:
        sep = '?'
        rest = link['href'].split(sep, 1)[0]
        links_new.append(rest)
    return(links_new)

# Returns url of UvA search engine with keywords
def create_search_query_url(keywords_list):
    main_url = "http://student.uva.nl/search?q=" + keywords_list[0]
    keywords_list.remove(keywords_list[0])
    
    for keyword in keywords_list:
        main_url = main_url + "+" + keyword
    
    return(main_url)

# Returns a list of keywords when given a question URL.
def get_keywords(question_url):
    # grab keywords with beautiful soup
    question_page = read_in_page(question_url)
    keyword_tag = question_page.findAll("meta", {"name": 'keywords'})
    keywords = keyword_tag[0]['content']
    return([keywords])

In [16]:
# Example test
query = 'Wat zijn de toelatingseisen van kunstmatige intelligentie?'
query_keywords = ['toelatingseisen', 'kunstmatige', 'intelligentie']
query_url = create_search_query_url(query_keywords)
print("Input query keywords:", query_keywords)
print("Query URL:", query_url, "\n")

query_page = read_in_page(query_url)
search_results = get_links(query_page)
top_five = search_results[:5]
print("Top 5 search results:", top_five, "\n")

print("Correct webpage URL:", search_results[0])
keywords = get_keywords(search_results[0])
print("Keywords of correct URL:", keywords)

Input query keywords: ['kunstmatige', 'intelligentie']
Query URL: http://student.uva.nl/search?q=toelatingseisen+kunstmatige+intelligentie 

Top 5 search results: ['http://www.uva.nl/programmas/bachelors/kunstmatige-intelligentie/toelating-en-inschrijven/toelatingseisen/toelatingseisen.html', 'http://www.uva.nl/shared-content/studentensites/fnwi/iw-gedeelde-content/nl/az/vakaanmelding/vakaanmelding.html', 'http://www.uva.nl/shared-content/studentensites/fnwi/iw-gedeelde-content/nl/az/bindend-studieadvies-bsa/bindend-studieadvies-bsa.html', 'http://www.student.uva.nl/ki', 'http://www.uva.nl/programmas/bachelors/kunstmatige-intelligentie/toelating-en-inschrijven/toelating-en-inschrijven.html'] 

Correct webpage URL: http://www.uva.nl/programmas/bachelors/kunstmatige-intelligentie/toelating-en-inschrijven/toelatingseisen/toelatingseisen.html
Keywords of correct URL: ['faculteiten,natuurwetenschappen, wiskunde en informatica']
